<a href="https://colab.research.google.com/github/vchiang001/DeepLabCut/blob/master/Using_DeepLabCut_screen_camera_trap_images_with_physical_interactions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Using DeepLabCut to filter out MegaDetector analysed camera trap images that show physically interacting animals**

![alt text](https://images.squarespace-cdn.com/content/v1/57f6d51c9f74566f55ecf271/1616492373700-PGOAC72IOB6AUE47VTJX/ke17ZwdGBToddI8pDm48kB8JrdUaZR-OSkKLqWQPp_YUqsxRUqqbr1mOJYKfIPR7LoDQ9mXPOjoJoqy81S2I8N_N4V1vUb5AoIIIbLZhVYwL8IeDg6_3B-BRuF4nNrNcQkVuAT7tdErd0wQFEGFSnBqyW03PFN2MN6T6ry5cmXqqA9xITfsbVGDrg_goIDasRCalqV8R3606BuxERAtDaQ/modelzoo.png?format=1000w)

The understanding of sexual diversity including the sociosexual interactions between organisms requires consideration of the natural milieu of these behaviours , as well as the evolutionary perspectives by comparing across species. Camera trap data is pivotal for this purpose to look at these behaviours in wildlife animals, and to understand their context (e.g. where and when). For more details, you can refer to this presentation during the 2022 DeepLabCut AI Residency Program at EPFL: https://zenodo.org/record/7040375#.YyJ9a3bMK39 

Our notebook runs on camera trap images that have been analysed using MegaDetector. More details on MegaDetector here: https://github.com/microsoft/CameraTraps/blob/main/megadetector.md 

Our notebook helps you isolate camera trap images that exhibit physical interactions. This is helpful because scientists invest a huge amount of time reviewing camera trap images, and a huge amount of that time is spent reviewing images they aren’t interested in (e.g. empty images, noise). For those interested in sexual behaviour, this can filter out a substantial amount of images, to visually inspect those images with physical interaction, whether sexual behaviour could be occurring. 

For this work, we will use DeepLabCut from the model zoo. More details here: http://modelzoo.deeplabcut.org Please cite the relevant papers for the models you use, and consider giving back by helping to label more data. More details here: https://contrib.deeplabcut.org/ 

- **What you need:**camera trap images, and the JSON output file from MegaDetector analysis (ensure your camera trap images retain the same name in the JSON output file)

- **What to do:** (1) in the top right corner, click "CONNECT". Then, just hit run (play icon) on each cell below and follow the instructions!


## **Let's get going: install DeepLabCut into COLAB:**
*Also, be sure you are connected to a GPU: go to menu, click Runtime > Change Runtime Type > select "GPU"*

In [ ]:
#click the play icon (this will take a few minutes to install all the dependences!)
!pip install deeplabcut-live
!pip install deeplabcut

**(Be sure to click "RESTART RUNTIME" if it is displayed above before moving on !)**

## Now let's set the backend & import the DeepLabCut package:

In [ ]:
import os

# stifle tensorflow warnings, like we get it already.
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

import deeplabcut

##Link your Google Drive (containing camera trap images & JSON file):

### First, place your camera trap image folder & JSON file into you google drive! 

In [ ]:
#Now, let's link to your GoogleDrive. Run this cell and follow the authorization instructions:

from google.colab import drive
drive.mount('/content/drive')

YOU WILL NEED TO EDIT THE FILE PATH & JSON PATH TO BE SET TO YOUR GOOGLE DRIVE LINK!

Typically, this will be: /content/drive/My Drive/CameraTrapImages


In [ ]:
#Setup your project variables:
# PLEASE EDIT THESE:
file_path = '/content/drive/My Drive/CameraTrapImages/'
json_path = '/content/drive/My Drive/CameraTrapImages/megadetector.json'

## (OPTIONAL) Pre-filter camera trap images
If you have thousands of camera trap images, you may want to pre-filter them depending on your needs. For our purposes, we want only camera trap images that have 2 or more animals, with MegaDetector confidence above a certain threshold. Running this cell will screen through all the images in your MegaDetector JSON output file, and COPY the images fulfilling our condition into a separate folder. *It will not delete any of your existing images only COPY to a new folder.


In [ ]:
#Please edit these:
mega_thresh = 0.6 #you can set it at a number between 0.0 – 1.0 (the higher the more confident)
filter_folder = "mre2anim" #what name you want to call this folder, this will be a new folder in your file_path

In [ ]:
import json 
import shutil
import os

# access your Megadetector JSON file
with open(json_path, 'r') as f:
    detection_results = json.load(f)

#creates a new folder for your filtered images
mre2anim = os.path.join(file_path, filter_folder)
os.mkdir(mre2anim) 

#if there are more than 2 animals that are above confidence thresholder
#the image will be copied to the folder defined above.
for img_data in detection_results['images']: 
    if len(img_data['detections']) >= 2: 
        anim_detect = 0 
        print(img_data['file'])
        for detect in img_data['detections']: 
            if detect['category'] == '1' and detect['conf'] >= mega_thresh:
                anim_detect += 1
        if anim_detect >= 2: 
            file = img_data['file'] 
            shutil.copy(file_path+'/'+file, mre2anim)  

## Select your model from the dropdown menu and set the model for analysis

In [ ]:
import ipywidgets as widgets
from IPython.display import display

model_options = deeplabcut.create_project.modelzoo.Modeloptions
model_selection = widgets.Dropdown(
    options=model_options,
    value=model_options[0],
    description="Choose a DLC ModelZoo model!",
    disabled=False
)
display(model_selection)

In [ ]:
model2use = model_selection.value
from deeplabcut.utils import auxfun_models
auxfun_models.download_model(modelname = model2use, target_dir = file_path)

YOU WILL NEED TO EDIT THE MODEL PATH & POSE CONFIG PATH TO BE SET TO YOUR GOOGLE DRIVE LINK!

This is currently set in your "file_path", so typically, this will be: /content/drive/My Drive/CameraTrapImages

In [ ]:
#Setup where your model is
# PLEASE EDIT THESE:
model_path = '/content/drive/My Drive/CameraTrapImages'
pose_cfg_path = model_path + '/pose_cfg.yaml' #copy the path to the pose_cfg.yaml 

## This function crops the bounding box from the JSON output

In [ ]:
import PIL
from PIL import Image, ImageFile, ImageFont, ImageDraw
import numpy as np
from tensorflow.python.eager.context import global_seed
def crop_detections(detect, img, imageWidth, imageHeight):
    x1, y1,w_box, h_box = detect["bbox"]
    ymin,xmin,ymax, xmax = y1, x1, y1 + h_box, x1 + w_box 
    area = (xmin * imageWidth, 
            ymin * imageHeight, 
            xmax * imageWidth,
            ymax * imageHeight)
    cropping = img.crop(area)
    list_np_crop = np.asarray(cropping)
    return list_np_crop 

## This function runs the DeepLabCut model on individual MegaDetector bounding box

In [ ]:
from dlclive import DLCLive, Processor

def predict_dlc(model_path, list_np_crop):
    #list_np_crop = np.array(list_np_crop)
    dlc_proc = Processor()
    dlc_live = DLCLive(model_path, processor=dlc_proc)
    dlc_live.init_inference(list_np_crop)
    keypts_xyp = dlc_live.get_pose(list_np_crop) # third column is llk!
    return keypts_xyp


## This function converts the DeepLabCut poses normalised to the original image

In [ ]:
import yaml
from ruamel.yaml import YAML
import pandas as pd 
def pose_orig_img(keypts_xyp, detect, pose_cfg_path):
    x1, y1, w_box, h_box = detect["bbox"] 
    keypts_xyp_orig_img = []
    for x, y, p in keypts_xyp:
        c = [x+x1, y+y1] 
        keypts_xyp_orig_img.append(c)
    with open(pose_cfg_path, "r") as stream:
        pose_cfg_dict = yaml.safe_load(stream)
    bodypart_label = pose_cfg_dict['all_joints_names']
    keypts_xyp_orig_img_df = pd.DataFrame (keypts_xyp_orig_img, columns = ['x', 'y'])
    keypts_xyp_orig_img_df['bodypart'] = bodypart_label
    keypts_xyp_orig_img_df = keypts_xyp_orig_img_df.set_index('bodypart')
    return keypts_xyp_orig_img_df

## This function obtains the reference length for each image (as each image differ in size) 

In [ ]:
#Please edit these:
mega_thresh = 0.6 #for your megadetector bounding box, you can set it at a number between 0.0 – 1.0 (the higher the more confident)
dlc_thresh = 0.001 #for your dlc poses, you can set it at a number between 0.0 – 1.0 (the higher the more confident)

In [ ]:
from scipy.spatial import distance
from statistics import mean

def nose2eye(allposes_names):
    global ref
    nose2eye_dist = [] 
    for entity in allposes_names: 
        eye_list = [] 
        df_body = globals()[entity]
        df_body_filt = df_body.loc[df_body['p'] > dlc_thresh] 
        for body in df_body_filt.index: 
            if body in eye_names: 
                eye_list.append(list(df_body_filt.loc[body])) 
        if len(eye_list) != 0: 
            eye_list = eye_list[0]
            eye_list = eye_list[:2]
            nose_list = [] 
            for body in df_body_filt.index:
                if body in nose_names:
                    nose_list.append(list(df_body_filt.loc[body]))
            if len(nose_list) != 0:
                nose_list = nose_list[0] 
                nose_list = nose_list[:2]
                dist = distance.euclidean(eye_list,nose_list)
                nose2eye_dist.append(dist)
    if len(nose2eye_dist) != 0:
        ref = min(nose2eye_dist)
    else:
        ref = 'no' 
    return ref 

## This function checks if there is physical interaction between individual MegaDetector boxes in each image

In [ ]:
import pandas as pd 
def phys_int(allposes_names, ref):
    int_matrx = [] 
    for entity in allposes_names:
        df_body = globals()[entity]
        df_body_filt = df_body.loc[df_body['p'] > dlc_thresh]
        df_body_filt = df_body_filt.drop(columns="p")
        xy = df_body_filt.values.tolist() 
        for entity in allposes_names: 
            df_body2 = globals()[entity]
            df_body_filt2 = df_body2.loc[df_body2['p'] > dlc_thresh]
            df_body_filt2 = df_body_filt2.drop(columns="p")
            ab = df_body_filt2.values.tolist() 
            for i in xy: 
                for b in ab: 
                    dist = distance.euclidean(i, b)
                    if dist != 0: 
                        int_matrx.append(dist) 
    if any(i < ref for i in int_matrx): 
        social = "physically interacting"
    else:
        social = 'not physically interacting'
    return social 

## After loading all the functions, we are ready to run! 

This creates a separate folder, and all camera trap images that exhibit physical interactions will be copied into the folder. \
Currently this is set to only the MegaDetector bounding boxes with confidence above 0.6, but this can be adjusted. \
Note that if you have thousands of images, it could take hours to run this (~10h for 7000 images). Therefore there is a textfile that keeps track which files and how many have been processed out of the total. There is an option below if your COLAB crashes, and you need to pick up from where it crashed. 


In [ ]:
# Please edit this:
camtrap4dlc = '/content/drive/My Drive/CameraTrapImages/mre2anim' #if you didn't do the optional filter step above, you need to change this 

In [ ]:
# open json file
import json 
with open(json_path, 'r') as f:
    detection_results = json.load(f)

# this is used to temporary name bounding boxes
import random 
import string  

# creates a new folder to copy camera trap images with physical interactions
import glob
import shutil
import os
phys_int_path = os.path.join(file_path, "phys_int_camtrap")
os.mkdir(phys_int_path) 

#finds the total number of images to process & writes to a new text file to monitor progress
num_img = len(os.listdir(camtrap4dlc))
txt_path = os.path.join(file_path, 'progress.txt')
prog_txt = open(txt_path, 'a')
prog_txt.write('total number of images is' + str(num_img))
prog_txt.close()

#bodypart names in various models to use for reference length 
eye_names = ['L_Eye', 'R_Eye', 'r_eye', 'l_eye', 'forehead', 'left_eye', 'right_eye']  
nose_names = ['Nose', 'nose', 'chin', 'upper_jaw', 'mouth_end_right ', 'lower_jaw', 'mouth_end_left']     


# this runs through all images in the JSON file, including monitoring the progress
# the loop goes through in order of the Megadetector JSON file
prog = 0 
prog_txt = open(txt_path, 'a') 
for img_data in detection_results["images"]:
    img_path =  os.path.join(file_path, img_data['file'])
    if os.path.exists(img_path) == True: 
      img = Image.open(img_path)
      imageWidth = img.size[0] 
      imageHeight = img.size[1]
      prog += 1 
      prog_txt.write(img_path + '\n') 
      prog_txt.write(str(prog) + 'out of' + str(num_img) + 'has been processed' + '\n')       
      if len(img_data["detections"]) >= 2: 
        print(img_data['file'])
        allposes_names = []  
        for detect in img_data["detections"]: 
            if detect['conf'] >= mega_thresh: 
                list_np_crop = crop_detections(detect, img, imageWidth, imageHeight) 
                keypts_xyp = predict_dlc(model_path, list_np_crop) 
                keypts_xyp_orig_img_df = pose_orig_img(keypts_xyp, detect, pose_cfg_path)
                letters = string.ascii_letters 
                name =  ''.join(random.choice(letters) for i in range(10)) 
                allposes_names.append(name) 
                locals()[name] = keypts_xyp_orig_img_df 
        ref =  nose2eye(allposes_names)
        if ref == 'no':
            print('no ref')
        else: 
            print(ref)
            social = phys_int(allposes_names, ref)
            file = img_data['file'] 
            if social == "physically interacting":
                shutil.copy(file_path+'/'+file, phys_int_path) 
prog_txt.close() 

## (OPTIONAL) Resuming if your COLAB crashes
Since the COLAB may need to run for a long time if you have thousands of camera trap data to process through. You can run this cell, to find out the last image that was processed, and find the index in the original JSON file. 

In [ ]:
# find the last img processed 
with open(txt_path, 'r') as f:
    last_proc_img = f.readlines()[-2]

#get the basename, without the /n 
print(last_proc_img)
last_proc_img_base = os.path.basename(last_proc_img)
last_proc_img_base_wo_n = last_proc_img_base[:-1]
print(last_proc_img_base_wo_n)

# finds the progress through the images 
with open(txt_path, 'r') as f:
    last_prog_line = f.readlines()[-1]

last_prog = int(last_prog_line[0])
print(last_prog)

#access the megadetector JSON file 
import json 
with open(json_path, 'r') as f:
    detection_results = json.load(f)

#find the index of a file 
files = detection_results['images']
for file in files:
    if file['file'] == last_proc_img_base_wo_n:
        last_proc_img_index = int(files.index(file)) 

print(last_proc_img_index) 

After you ran the above cells, and finds the index, you can run the cell below. It's similar to the the one above, but it changes to start from the image with the index number found above. 

In [ ]:
import json 
import random 
import string  
import glob
import shutil
import os

with open(json_path, 'r') as f:
    detection_results = json.load(f)
phys_int_path = os.path.join(file_path, "phys_int_camtrap")
num_img = len(os.listdir(camtrap4dlc))
txt_path = os.path.join(file_path, 'progress.txt')
prog = last_prog 

eye_names = ['L_Eye', 'R_Eye', 'r_eye', 'l_eye', 'forehead', 'left_eye', 'right_eye']  
nose_names = ['Nose', 'nose', 'chin', 'upper_jaw', 'mouth_end_right ', 'lower_jaw', 'mouth_end_left']    

prog_txt = open(txt_path, 'a') 
for img_data in detection_results["images"][last_proc_img_index:]: 
    img_path =  os.path.join(file_path, img_data['file'])
    if os.path.exists(img_path) == True: 
      img = Image.open(img_path)
      imageWidth = img.size[0] 
      imageHeight = img.size[1]
      prog += 1 
      prog_txt.write(img_path + '\n') 
      prog_txt.write(str(prog) + 'out of' + str(num_img) + 'has been processed' + '\n')       
      if len(img_data["detections"]) >= 2: 
        print(img_data['file'])
        allposes_names = []  
        for detect in img_data["detections"]: 
            if detect['conf'] >= mega_thresh: 
                list_np_crop = crop_detections(detect, img, imageWidth, imageHeight) 
                keypts_xyp = predict_dlc(model_path, list_np_crop) 
                keypts_xyp_orig_img_df = pose_orig_img(keypts_xyp, detect, pose_cfg_path)
                letters = string.ascii_letters 
                name =  ''.join(random.choice(letters) for i in range(10)) 
                allposes_names.append(name) 
                locals()[name] = keypts_xyp_orig_img_df 
        ref =  nose2eye(allposes_names)
        if ref == 'no':
            print('no ref')
        else: 
            print(ref)
            social = phys_int(allposes_names, ref)
            file = img_data['file'] 
            if social == "physically interacting":
                shutil.copy(file_path+'/'+file, phys_int_path) 
prog_txt.close() 

###Happy DeepLabCutting! Welcome to the Zoo :)